In [ ]:
# if the following command generates an error, you probably didn't enable
# the cluster security option "Allow API access to all Google Cloud services"
# under Manage Security → Project Access when setting up the cluster
!gcloud dataproc clusters list --region us-central1

NAME          PLATFORM  PRIMARY_WORKER_COUNT  SECONDARY_WORKER_COUNT  STATUS   ZONE           SCHEDULED_DELETE
cluster-7eaf  GCE       4                                             RUNNING  us-central1-a


In [ ]:
!pip install -q google-cloud-storage==1.43.0
!pip install -q graphframes

In [ ]:
import pyspark
import sys
from collections import Counter, OrderedDict, defaultdict
import itertools
from itertools import islice, count, groupby
import pandas as pd
import os
import re
from operator import itemgetter
import nltk
from nltk.stem.porter import *
from nltk.corpus import stopwords
from time import time
from pathlib import Path
import pickle
import pandas as pd
from google.cloud import storage

import hashlib
def _hash(s):
    return hashlib.blake2b(bytes(s, encoding='utf8'), digest_size=5).hexdigest()

nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [ ]:
# if nothing prints here you forgot to include the initialization script when starting the cluster
!ls -l /usr/lib/spark/jars/graph*

-rw-r--r-- 1 root root 247882 Feb 29 15:30 /usr/lib/spark/jars/graphframes-0.8.2-spark3.1-s_2.12.jar


In [ ]:
from pyspark.sql import *
from pyspark.sql.functions import *
from pyspark import SparkContext, SparkConf, SparkFiles
from pyspark.sql import SQLContext
from graphframes import *

In [ ]:
spark

In [ ]:
# Put your bucket name below and make sure you can access it without an error
bucket_name = 'wikiproject-414111-bucket'
full_path = f"gs://{bucket_name}/"
paths=[]

client = storage.Client()
blobs = client.list_blobs(bucket_name)
for b in blobs:
    if b.name != 'graphframes.sh' and '.parquet' in b.name:
        paths.append(full_path+b.name)

In [ ]:
parquetFile = spark.read.parquet(*paths)
doc_text__title_anchor_quartet = parquetFile.select("text", "id", "title", "anchor_text").rdd

In [ ]:
# Count number of wiki pages
num_docs = parquetFile.count()

6348910

In [ ]:
# if nothing prints here you forgot to upload the file inverted_index_gcp.py to the home dir
%cd -q /home/dataproc
!ls inverted_index_gcp.py

inverted_index_gcp.py


In [ ]:
# adding our python module to the cluster
sc.addFile("/home/dataproc/inverted_index_gcp.py")
sys.path.insert(0,SparkFiles.getRootDirectory())

In [ ]:
from inverted_index_gcp import *

In [ ]:
english_stopwords = frozenset(stopwords.words('english'))
corpus_stopwords = ["category", "references", "also", "external", "links",
                    "may", "first", "see", "history", "people", "one", "two",
                    "part", "thumb", "including", "second", "following",
                    "many", "however", "would", "became"]

all_stopwords = english_stopwords.union(corpus_stopwords)
RE_WORD = re.compile(r"""[\#\@\w](['\-]?\w){2,24}""", re.UNICODE)

porter_stemmer = PorterStemmer()

bucket_name = 'wikiproject-414111-bucket'

NUM_BUCKETS = 124
def token2bucket_id(token):
  return int(_hash(token),16) % NUM_BUCKETS

# PLACE YOUR CODE HERE
def word_count(text, id):
  ''' Count the frequency of each word in `text` (tf) that is not included in
  `all_stopwords` and return entries that will go into our posting lists.
  Parameters:
  -----------
    text: str
      Text of one document
    id: int
      Document id
  Returns:
  --------
    List of tuples
      A list of (token, (doc_id, tf)) pairs
      for example: [("Anarchism", (12, 5)), ...]
  '''
  tokens = [porter_stemmer.stem(token.group()) for token in RE_WORD.finditer(text.lower())]
  stop_tokens = set(tokens).intersection(all_stopwords)
  c = Counter(t for t in tokens if t not in stop_tokens)
  # Divide by length of document
  c = {k : (v / len(c)) for k,v in c.items()}
  count_list = []
  for token, tf in c.items():
    count_list.append((token, (id, tf)))
  return count_list

def word_count_title(title, id):
  ''' Count the frequency of each word in `title` (tf) that is not included in
  `all_stopwords` and return entries that will go into our posting lists.
  Parameters:
  -----------
    title: str
      Title of one document
    id: int
      Document id
  Returns:
  --------
    List of tuples
      A list of (token, (doc_id, tf)) pairs
      for example: [("Anarchism", (12, 5)), ...]
  '''
  tokens = [porter_stemmer.stem(token.group()) for token in RE_WORD.finditer(title.lower())]
  stop_tokens = set(tokens).intersection(all_stopwords)
  c = Counter(t for t in tokens if t not in stop_tokens)
  # Divide by length of document
  c = {k : (v / len(c)) for k,v in c.items()}
  count_list = []
  for token, tf in c.items():
    count_list.append((token, (id, tf)))
  return count_list

def word_count_anchor(anchor, id):
  ''' Count the frequency of each word in `anchor` (tf) that is not included in
  `all_stopwords` and return entries that will go into our posting lists.
  Parameters:
  -----------
    anchor: str
      Anchor of one document
    id: int
      Document id
  Returns:
  --------
    List of tuples
      A list of (token, (doc_id, tf)) pairs
      for example: [("Anarchism", (12, 5)), ...]
  '''
  tokens = [porter_stemmer.stem(token.group()) for token in RE_WORD.finditer(anchor.lower())]
  # YOUR CODE HERE
  stop_tokens = set(tokens).intersection(all_stopwords)
  c = Counter(t for t in tokens if t not in stop_tokens)
  # Divide by length of document
  c = {k : (v / len(c)) for k,v in c.items()}
  count_list = []
  for token, tf in c.items():
    count_list.append((token, (id, tf)))
  return count_list


def reduce_word_counts(unsorted_pl):
  ''' Returns a sorted posting list by wiki_id.
Parameters:
-----------
  unsorted_pl: list of tuples
    A list of (wiki_id, tf) tuples
Returns:
--------
  list of tuples
    A sorted posting list.
  '''
  return sorted(unsorted_pl, key=lambda x: x[0])

def create_title_id_dict(doc_text_pairs, base_dir, name):
  # Initialize an empty dictionary
  id_title = doc_text_pairs.flatMap(lambda x: [(x['id'], x['title'])])

  # Write dictionary to file
  path = str(Path(base_dir) / f'{name}.pkl')
  bucket = None if bucket_name is None else get_bucket(bucket_name)
  with _open(path, 'wb', bucket) as f:
      pickle.dump(id_title.collectAsMap(), f)

def calculate_df(postings):
  ''' Takes a posting list RDD and calculate the df for each token.
  Parameters:
  -----------
    postings: RDD
      An RDD where each element is a (token, posting_list) pair.
  Returns:
  --------
    RDD
      An RDD where each element is a (token, df) pair.
  '''
  # YOUR CODE HERE
  return postings.flatMap(lambda x: [(x[0], len(x[1]))])

def calculate_tf_idf(postings, num_docs):
  ''' Takes a posting list RDD and calculate the df for each token.
  Parameters:
  -----------
    postings: RDD
      An RDD where each element is a (token, posting_list) pair.
  Returns:
  --------
    RDD
      An RDD where each element is a (token, [(doc_id, tf_idf), (doc_id, tf_idf)...] pairs.
  '''
  # YOUR CODE HERE
  return postings.flatMap(lambda x: [(x[0], [(post[0], (post[1] * np.log10(num_docs / len(x[1])))) for post in x[1]])])

def calculateDocLength(postings, base_dir, name):
  ''' Takes a posting list RDD, calculates the doc length (norm) of each doc and puts in dictionary.
  saves the doc length as pickle file.
  Parameters:
  -----------
    postings: RDD
      An RDD where each element is a (token, posting_list) pair.
  '''
  docs = postings.flatMap(lambda x: (x[1]))
  docs_id = docs.groupByKey().mapValues(lambda x: list(x))
  docs_lengths = docs_id.flatMap(lambda x: [(x[0], np.sqrt(np.sum([np.power(tf_idf, 2) for tf_idf in x[1]])))])

  path = str(Path(base_dir) / f'{name}.pkl')
  bucket = None if bucket_name is None else get_bucket(bucket_name)
  with _open(path, 'wb', bucket) as f:
      pickle.dump(docs_lengths.collectAsMap(), f)

def partition_postings_and_write(postings, base_dir):
  ''' A function that partitions the posting lists into buckets, writes out
  all posting lists in a bucket to disk, and returns the posting locations for
  each bucket. Partitioning should be done through the use of `token2bucket`
  above. Writing to disk should use the function  `write_a_posting_list`, a
  static method implemented in inverted_index_colab.py under the InvertedIndex
  class.
  Parameters:
  -----------
    postings: RDD
      An RDD where each item is a (w, posting_list) pair.
  Returns:
  --------
    RDD
      An RDD where each item is a posting locations dictionary for a bucket. The
      posting locations maintain a list for each word of file locations and
      offsets its posting list was written to. See `write_a_posting_list` for
      more details.
  '''
  buckets_id = postings.flatMap(lambda x: [(token2bucket_id(x[0]), (x[0], x[1]))])
  grouped_buckets = buckets_id.groupByKey().mapValues(lambda x: list(x))
  posting_locs = grouped_buckets.flatMap(lambda x: [InvertedIndex.write_a_posting_list(x, base_dir, bucket_name)])
  return posting_locs

In [ ]:
# create title-id dict and write out
create_title_id_dict(doc_text__title_anchor_quartet, '.', "title_id")

# Build Text Index

In [ ]:
word_counts_text = doc_text_title_three.flatMap(lambda x: word_count(x[0], x[1]))
postings_text = word_counts_text.groupByKey().mapValues(reduce_word_counts)
# filtering postings and calculate df
postings_text_filtered = postings_text.filter(lambda x: len(x[1])>50)
tf_idf_postings_text = calculate_tf_idf(postings_text_filtered, num_docs)
w2df = calculate_df(postings_text_filtered)
w2df_dict = w2df.collectAsMap()
# partition posting lists and write out
_ = partition_postings_and_write(postings_text_filtered, 'text_postings').collect()

In [ ]:
# collect all posting lists locations into one super-set
super_posting_text_locs = defaultdict(list)
for blob in client.list_blobs(bucket_name, prefix='text_postings'):
  if not blob.name.endswith("pickle"):
    continue
  with blob.open("rb") as f:
    posting_locs = pickle.load(f)
    for k, v in posting_locs.items():
      super_posting_text_locs[k].extend(v)

In [ ]:
# Create inverted index instance
inverted_text = InvertedIndex()
# Adding the posting locations dictionary to the inverted index
inverted_text.posting_locs = super_posting_locs
# Add the token - df dictionary to the inverted index
inverted_text.df = w2df_dict
# write the global stats out
inverted_text.write_index('text_postings', 'index_text', bucket_name)
# upload to gs
index_src = "index_text.pkl"
index_dst = f'gs://{bucket_name}/text_postings/{index_src}'
!gsutil cp $index_src $index_dst

In [ ]:
!gsutil ls -lh $index_dst

In [ ]:
# calculate docs lengths (norms) and write out
calculateDocLength(tf_idf_postings_text, '.', "text_lengths")

# Build Title Index

In [ ]:
word_counts_title = doc_text__title_anchor_quartet.flatMap(lambda x: word_count_title(x[2], x[1]))
postings_title = word_counts_title.groupByKey().mapValues(reduce_word_counts)
tf_idf_postings_title = calculate_tf_idf(postings_title, num_docs)
# filtering postings and calculate df
w2df = calculate_df(postings_title)
w2df_dict = w2df.collectAsMap()
# partition posting lists and write out
_ = partition_postings_and_write(tf_idf_postings_title, 'title_postings').collect()

NameError: name 'num_docs' is not defined

In [ ]:
# collect all posting lists locations into one super-set
super_posting_title_locs = defaultdict(list)
for blob in client.list_blobs(bucket_name, prefix='title_postings'):
  if not blob.name.endswith("pickle"):
    continue
  with blob.open("rb") as f:
    posting_locs = pickle.load(f)
    for k, v in posting_locs.items():
      super_posting_title_locs[k].extend(v)

In [ ]:
# Create inverted index instance
inverted_title = InvertedIndex()
# Adding the posting locations dictionary to the inverted index
inverted_title.posting_locs = super_posting_title_locs
# Add the token - df dictionary to the inverted index
inverted_title.df = w2df_dict
# write the global stats out
inverted_title.write_index('.', 'index_title', bucket_name)
# upload to gs
index_src = "index_title.pkl"
index_dst = f'gs://{bucket_name}/title_postings/{index_src}'
!gsutil cp $index_src $index_dst

CommandException: No URLs matched: index_title.pkl


In [ ]:
!gsutil ls -lh $index_dst

 62.74 MiB  2024-02-29T15:55:46Z  gs://wikiproject-414111-bucket/title_postings/index_title.pkl
TOTAL: 1 objects, 65785858 bytes (62.74 MiB)


In [ ]:
# calculate docs lengths (norms) and write out
calculateDocLength(tf_idf_postings_title, '.', "title_lengths")

In [ ]:
test_index = InvertedIndex.read_index('.', 'index_title', bucket_name)

In [ ]:
print(inverted_title.read_a_posting_list('.', 'american', bucket_name))

[(309, 0.5), (615, 0.3333333333333333), (659, 0.25), (771, 0.3333333333333333), (863, 0.3333333333333333), (869, 0.3333333333333333), (966, 0.5), (1110, 0.3333333333333333), (1111, 0.3333333333333333), (1112, 0.3333333333333333), (1201, 0.3333333333333333), (1241, 0.5), (1338, 0.25), (1488, 0.5), (1558, 0.3333333333333333), (1890, 0.5), (1902, 0.3333333333333333), (1950, 0.25), (1973, 0.5), (2011, 0.25), (2154, 0.5), (2175, 0.5), (2357, 0.3333333333333333), (2386, 0.5), (2472, 0.3333333333333333), (2799, 0.25), (2856, 0.25), (3861, 0.25), (3862, 0.25), (4166, 0.2), (4218, 0.25), (9734, 0.5), (21217, 0.25), (21673, 0.3333333333333333), (21976, 0.3333333333333333), (21981, 0.25), (22050, 0.2), (22788, 0.3333333333333333), (24710, 0.25), (28265, 0.3333333333333333), (29507, 0.5), (34191, 0.5), (35922, 0.25), (37032, 0.2), (40673, 0.5), (42042, 0.3333333333333333), (42248, 0.5), (42899, 0.3333333333333333), (42905, 0.2), (44122, 0.25), (45929, 0.25), (46884, 0.3333333333333333), (47571, 0.

# Build Anchor Index

In [ ]:
word_counts_anchor = doc_text__title_anchor_quartet.flatMap(lambda x: word_count_title(x[3], x[1]))
postings_anchor = word_counts_anchor.groupByKey().mapValues(reduce_word_counts)
tf_idf_postings_anchor = calculate_tf_idf(postings_anchor, num_docs)
# filtering postings and calculate df
w2df = calculate_df(postings_anchor)
w2df_dict = w2df.collectAsMap()
# partition posting lists and write out
_ = partition_postings_and_write(tf_idf_postings_anchor, 'anchor_postings').collect()

In [ ]:
# collect all posting lists locations into one super-set
super_posting_anchor_locs = defaultdict(list)
for blob in client.list_blobs(bucket_name, prefix='anchor_postings'):
  if not blob.name.endswith("pickle"):
    continue
  with blob.open("rb") as f:
    posting_locs = pickle.load(f)
    for k, v in posting_locs.items():
      super_posting_anchor_locs[k].extend(v)

In [ ]:
# Create inverted index instance
inverted_anchor = InvertedIndex()
# Adding the posting locations dictionary to the inverted index
inverted_anchor.posting_locs = super_posting_anchor_locs
# Add the token - df dictionary to the inverted index
inverted_anchor.df = w2df_dict
# write the global stats out
inverted_anchor.write_index('.', 'index_anchor', bucket_name)
# upload to gs
index_src = "index_anchor.pkl"
index_dst = f'gs://{bucket_name}/anchor_postings/{index_src}'
!gsutil cp $index_src $index_dst

CommandException: No URLs matched: index_title.pkl


In [ ]:
!gsutil ls -lh $index_dst

In [ ]:
# calculate docs lengths (norms) and write out
calculateDocLength(tf_idf_postings_anchor, '.', "anchor_lengths")